In [95]:
import pandas as pd
import numpy as np

def main():
    district, districtGeometry, districtType, districtRatings, districtAEA, \
        districtDistinctions, districtReference, districtECHS = readDistrictFiles()
    district = combineDistrictFiles(district, districtGeometry, districtType, districtRatings, districtAEA,
                                    districtDistinctions, districtReference, districtECHS)

def readDistrictFiles():
    #read in csv files
    district = pd.read_csv('tea directory districts.csv', usecols= {0,1,2,3,4,14}, dtype={'DISTRICT_NUMBER': object, 
                                                                                              'COUNTY_NUMBER': object, 
                                                                                              'REGION_NUMBER': object})
    districtGeometry = pd.read_csv('district_geometry.csv', usecols={'DISTRICT_N','Area'}, dtype={'DISTRICT_N': object})
    districtGeometry['DISTRICT_N']=fixZeros(list(districtGeometry['DISTRICT_N']))
    districtType = pd.read_csv('district type.csv', usecols = {1,2,3}, dtype={'District': object})
    districtRatings = pd.read_csv('district ratings.csv', dtype={'District_Number': object})   
    districtAEA = pd.read_csv('aea districts.csv', usecols = {1}, dtype={'District Number': object})
    
    #read in xlsx files
    districtDistinctions = pd.read_excel('district_distinctions.xlsx', sheetname=0, converters={'DISTRICT':str})
    districtReference = pd.read_excel('district_reference.xlsx', sheetname=0, converters={'DISTRICT':str})
    districtECHS = pd.read_excel('Early College High Schools 15-16 list.xlsx', sheetname=0, converters={'DistrictNumber':str,
                                                                                                            'CampusNumber':str})
    #get unique values from campus ECHS list as new object
    #wasn't sure if there was much benefit to having a one column data frame or a series, did series
    districtECHS = pd.DataFrame({'DISTRICTN':districtECHS.DistrictNumber.unique(), 'ECHS_FLAG':'Y'}) 

    #add flags
    districtAEA['AEA_FLAG'] = 'Y'
    districtDistinctions['DISTINCTION_FLAG'] = np.where(districtDistinctions['DAD_POST']=='1','Y','N')
    
    #remove extra columns
    districtRatings.drop(districtRatings.columns[[0]], axis=1, inplace=True)
    districtReference = districtReference[['DISTRICT','DFLCHART','DI1_MET','DI1','DI2_MET','DI2','DI3_MET',
                                           'DI3','DI4_MET','DI4']]
    districtDistinctions = districtDistinctions[['DISTRICT','DISTINCTION_FLAG']]
    
    #rename index columns to standard name
    district = district.rename(columns = {'DISTRICT_NUMBER':'DISTRICT_N'})
    districtGeometry = districtGeometry.rename(columns = {'DISTRICT_N':'DISTRICT_N'})   
    districtType = districtType.rename(columns = {'District':'DISTRICT_N'})
    districtRatings = districtRatings.rename(columns = {'District_Number':'DISTRICT_N'})
    districtAEA = districtAEA.rename(columns = {'District Number':'DISTRICT_N'})    
    districtDistinctions = districtDistinctions.rename(columns = {'DISTRICT':'DISTRICT_N'})
    districtReference = districtReference.rename(columns = {'DISTRICT':'DISTRICT_N'})
    
    #rename other columns
    districtType = districtType.rename(columns = {'Description':'Type_Description'})
    districtReference = districtReference.rename(columns = {'DFLCHART':'CHARTER_OPERATOR'})
    
    #set index columns
    district = district.set_index('DISTRICT_N')
    districtGeometry = districtGeometry.set_index('DISTRICT_N')   
    districtType = districtType.set_index('DISTRICT_N') 
    districtRatings = districtRatings.set_index('DISTRICT_N')  
    districtAEA = districtAEA.set_index('DISTRICT_N')    
    districtDistinctions = districtDistinctions.set_index('DISTRICT_N')
    districtReference = districtReference.set_index('DISTRICT_N')
    
    return district, districtGeometry, districtType, districtRatings, districtAEA, \
        districtDistinctions, districtReference, districtECHS
    
def combineDistrictFiles(district, districtGeometry, districtType, districtRatings, districtAEA,
                                    districtDistinctions, districtReference, districtECHS):
    district = district.join(districtReference,how='left')
    district = district.join(districtGeometry,how='left')
    district = district.join(districtType,how='left')
    district = district.join(districtAEA,how='left')
    district = district.join(districtECHS,how='left')
    district = district.join(districtRatings,how='left')
    district = district.join(districtDistinctions,how='left')
    
    #convert all column names to upper case for cleanliness
    district.columns = district.columns.str.upper()
    print(district)
    
    
def fixZeros(str):
    #pass a list of numbers formatted as string and fill in the missing leading zeros
    maxlen = max(len(i) for i in str)
    for i in range(len(str)):
        if len(str[i]) < maxlen:
            str[i] = "0"*(maxlen - len(str[i]))+str[i]
    return str
    
main()

           COUNTY_NUMBER REGION_NUMBER                 DISTRICT_NAME  \
DISTRICT_N                                                             
001902               001            07                    Cayuga Isd   
001903               001            07                   Elkhart Isd   
001904               001            07                 Frankston Isd   
001906               001            07                    Neches Isd   
001907               001            07                 Palestine Isd   
001908               001            07                  Westwood Isd   
001909               001            07                    Slocum Isd   
002901               002            18                   Andrews Isd   
003801               003            07  Pineywoods Community Academy   
003902               003            07                    Hudson Isd   
003903               003            07                    Lufkin Isd   
003904               003            07                Huntington